# Feature Engineering

Here we'll engineer features having to do w/ transactions.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set()

## Load card data

In [2]:
# Load card data
# no nulls in test/train except for ONE ROW in test! (first_active_month)
dtypes = {
  'card_id':            'str',     # 201917 unique vals
  'target':             'float32', # -33.22 thru ~18
  'first_active_month': 'str',     # 2011-10 thru 2018-02
  'feature_1':          'uint8',   # 1 thru 5
  'feature_2':          'uint8',   # 1 thru 3
  'feature_3':          'uint8',   # 0 and 1
}
train = pd.read_csv('../input/train.csv',
                    usecols=dtypes.keys(),
                    dtype=dtypes)
del dtypes['target']
test = pd.read_csv('../input/test.csv',
                   usecols=dtypes.keys(),
                   dtype=dtypes)

# Add target col to test
test['target'] = np.nan

# Merge test and train
cards = pd.concat([train, test])

As a quick sanity check, let's make sure there aren't any `card_id`s which are in both test and train:

In [3]:
print('Num unique in train:  ', test['card_id'].nunique())
print('Num unique in test:   ', train['card_id'].nunique())
print('The sum:              ', test['card_id'].nunique()+train['card_id'].nunique())
print('Num unique in merged: ', cards['card_id'].nunique())

Num unique in train:   123623
Num unique in test:    201917
The sum:               325540
Num unique in merged:  325540


OK good, there aren't.  Now we can delete the original dataframes.

In [4]:
del train, test
gc.collect()

7

Let's take a look at the cards data.

In [5]:
cards.sample(10)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
47654,2016-10,C_ID_6a94561348,5,2,1,NaN
118077,2015-11,C_ID_e6e618c6c5,3,1,1,0.951745
2774,2017-10,C_ID_2f1649ab1c,4,3,0,NaN
18644,2013-09,C_ID_53aa749f46,3,3,1,NaN
27549,2016-07,C_ID_5146da3e75,2,1,0,NaN
169702,2017-09,C_ID_feb16aac02,2,1,0,-1.095193
66495,2016-09,C_ID_3e930a1816,2,2,0,-0.339727
50094,2017-10,C_ID_f58790644c,2,2,0,NaN
5812,2017-08,C_ID_4a3f9c1a73,3,2,1,NaN
121239,2017-10,C_ID_8d778b799e,2,3,0,NaN


The `card_id`s always start with `C_ID_`.  That's kind of a waste of space...  All the `card_id`s are the same length:

In [6]:
cards['card_id'].apply(len).unique()

array([15])

If we cut off the prefix, the remaining strings appear to be all hexidecimal (represented by values 0-9 and a-f):

In [7]:
cards['card_id'].str.slice(5, 15).sample(10)

14034     35bcf05de0
130797    10df245054
110761    a617e2a8e4
43650     7a4ba00f51
176190    7f59c2c722
5596      ab05e7812d
66650     31d0aa05be
46314     a4220786ba
17973     f950a6e72f
42786     bad5624081
Name: card_id, dtype: object

We can check that every single entry is indeed a hexideximal:

In [8]:
(cards['card_id']
 .str.slice(5, 15)
 .apply(lambda x: all(e in '0123456789abcdef' for e in x))
 .all())

True

To save space, we could convert the `card_id`s to integers like this:

In [9]:
#cards['card_id'] = cards['card_id'].apply(lambda x: int(x, 16)).astype('uint64')

There are 5 bytes worth of hex info in the card ids, and the values span the full 5-byte range, so we'd have to use a 64-bit integer to represent them (instead of a 32-bit int, which is only 4 bytes).  BUT, there are only 201,917 unique `card_id`s in `train.csv` and 123,623 in `test.csv`, which can easily be represented by a `uint32` (which stores values up to ~4 billion).  The inneficiency of using a 64-bit representation for something where a 32-bit reprentation would do... Bothers me. \*eye twitches\*

To use a 32-bit integer, we'll create a map between the card_id and a unique integer which identifies it, and then map the string values to integer values.  (we need to create a map so that we can map the values in the transactions data in the exact same way)

In [10]:
# Create a map from card_id to unique int
card_id_map = dict(zip(
    cards['card_id'].values,
    cards['card_id'].astype('category').cat.codes.values
))

# Map the values
cards['card_id'] = cards['card_id'].map(card_id_map).astype('uint32')

Now our `card_id`s are 32-bit integers:

In [11]:
cards.sample(10)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
171188,2017-10,142954,5,1,1,0.127462
132976,2017-11,317706,2,1,0,0.299097
91580,2016-12,323420,3,2,1,NaN
42686,2016-12,4144,1,1,0,NaN
11942,2017-10,35124,2,1,0,-2.660734
121790,2016-07,127244,3,2,1,NaN
40540,2017-05,87499,3,2,1,-0.968913
149283,2017-03,312690,2,1,0,0.991489
201513,2015-11,135022,3,3,1,-0.233539
8970,2016-05,42913,3,3,1,0.365495


Next, we'll convert the `first_active_month` from a string to a datetime.  Unfortunately, there is one.  Single.  Row.  Where `first_active_month` is NaN.

In [12]:
cards[cards['first_active_month'].isnull()]

,first_active_month,card_id,feature_1,feature_2,feature_3,target
11578,NaN,247334,5,2,1,NaN


But other than that row, all the other `first_active_months` are in `YYYY-MM` format:

In [13]:
def nanlen(x):
    if type(x) is float:
        return 'NaN'
    else:
        return len(x)
    
cards['first_active_month'].apply(nanlen).unique()

array([7, 'NaN'], dtype=object)

So, we can convert `first_active_month` column to datetime format.

In [14]:
# Convert first_active_month to datetime
cards['first_active_month'] = pd.to_datetime(cards['first_active_month'],
                                             format='%Y-%m')

Finally, we'll set the index to be the `card_id`.

In [15]:
# Make card_id the index
cards.set_index('card_id', inplace=True)
gc.collect()
cards.sample(10)

,first_active_month,feature_1,feature_2,feature_3,target
card_id,,,,,
259006,2017-08-01,2,1,0,-1.227113
156228,2017-01-01,3,2,1,NaN
145590,2016-08-01,3,3,1,0.854417
78292,2017-11-01,3,2,1,1.025100
253311,2016-12-01,3,3,1,0.747700
210572,2016-03-01,2,2,0,1.033730
110490,2016-04-01,2,2,0,3.107079
243364,2017-09-01,2,3,0,1.586964
105237,2017-05-01,2,2,0,NaN


## Load Merchants Data

NOTE: some cols are in both merchants dataset and the transactions datasets, and the values differ.  Probably those are values which are properties of the merchants but have changed between the time the purchase was made and the time at which the merchants table was compiled.  So, for per-transaction information we'll use the version of the information in the transactions dataset, not the info from the merchants dataset.  (I.e., that's why we don't read some cols in below)

In [16]:
# Datatypes of each column
# (don't load cols which are in transactions data, just use those vals)
# Nulls: NO nulls except for 13 rows in avg_sales_lag{3,6,12}
dtypes = {
  'merchant_id':                 'str',     # 334633 unique values
  'merchant_group_id':           'uint32',  # 1 thru 112586 (w/ some missing, ~109k uniques)
  'numerical_1':                 'float32', # ~ -0.06 thru ~ 183.8 (only 951 unique vals?)
  'numerical_2':                 'float32', # roughly the same as above
  'most_recent_sales_range':     'str',     # A, B, C, D, or E
  'most_recent_purchases_range': 'str',     # A, B, C, D, or E
  'avg_sales_lag3':              'float32', # most between 0 and 2, if you transform by 1/x, all but 3 are between 0 and 4
  'avg_purchases_lag3':          'float32', # most between 0 and 2, if you transform by 1/x, all but 3 are between 0 and 4
  'active_months_lag3':          'uint8',   # 1 to 3 
  'avg_sales_lag6':              'float32', # similar to avg_sales_lag3
  'avg_purchases_lag6':          'float32', # similar to avg_purchases_lag3
  'active_months_lag6':          'uint8',   # 1 to 6
  'avg_sales_lag12':             'float32', # similar to avg_sales_lag3
  'avg_purchases_lag12':         'float32', # similar to avg_purchases_lag3
  'active_months_lag12':         'uint8',   # 1 to 12
  'category_4':                  'str',     # Y or N
}

# Load the data
merchants = pd.read_csv('../input/merchants.csv',
                        usecols=dtypes.keys(),
                        dtype=dtypes)

In [17]:
# Map merchant_id to integer
merch_id_map = dict(zip(
    merchants['merchant_id'].values,
    merchants['merchant_id'].astype('category').cat.codes.values
))

In [18]:
def preprocess_merch_data(df):
    
    # Convert merchant ID to numbers
    df['merchant_id'] = df['merchant_id'].map(merch_id_map).astype('float32')

    # Inverse transforms
    inversions = [
        'avg_sales_lag3',
        'avg_sales_lag6',
        'avg_sales_lag12',
        'avg_purchases_lag3',
        'avg_purchases_lag6',
        'avg_purchases_lag12',
    ]
    for col in inversions:
        df[col] = 1.0/df[col]

    # Encode categorical columns
    bool_map = {'Y': 1, 'N': 0}
    five_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
    conversions = [
        ('category_4', bool_map, 'uint8'),
        ('most_recent_sales_range', five_map, 'uint8'),
        ('most_recent_purchases_range', five_map, 'uint8')
    ]
    for col, mapper, new_type in conversions:
        df[col] = df[col].map(mapper).astype(new_type)
        
    # Clean up
    gc.collect()

# Preprocess the merchants data
preprocess_merch_data(merchants)

## Load Transactions Data

In [19]:
# Datatypes of each column
# only NaNs are in category_3, merchant_id, and category_2
dtypes = {
    'authorized_flag':      'str',     # Y or N
    'card_id':              'str',     # 325540 unique values
    'city_id':              'int16',   # -1 then 1 to 347 (is -1 supposed to be nan?)
    'category_1':           'str',     # Y or N
    'installments':         'int8',    # -25, then -1 thru 12 (-1 supposed to be nan?)
    'category_3':           'str',     # A, B, C, and nan (ordinal?)
    'merchant_category_id': 'int16',   # 2 to 891
    'merchant_id':          'str',     # 334633 unique values and nans (164697 nans!)
    'month_lag':            'int8',    # -13 thru 0
    'purchase_amount':      'float32', # min: -0.746, med: -0.699, max: 11269.667
    'purchase_date':        'str',     # YYYY-MM-DD hh:mm:ss
    'category_2':           'float32', # 1 thru 5 and nan (ordinal?)
    'state_id':             'int8',    # -1 then 1 thru 24
    'subsector_id':         'int8'     # 1 thru 41
}

# Load the data
hist_trans = pd.read_csv('../input/historical_transactions.csv', 
                         usecols=dtypes.keys(),
                         dtype=dtypes)
new_trans = pd.read_csv('../input/new_merchant_transactions.csv', 
                        usecols=dtypes.keys(),
                        dtype=dtypes)

We also need to convert the `card_id` to an integer as before, `merchant_id` to an integer in the same way, convert the `purchase_date` column (which has been loaded as a string) to datetime format, and encode the categorical columns.

In [20]:
def preprocess_trans_data(df):
    
    # Convert card_id and merchant_id to numbers
    df['card_id'] = df['card_id'].map(card_id_map).astype('uint32')
    df['merchant_id'] = df['merchant_id'].map(merch_id_map).astype('float32')

    # Convert purchase_date to datetime
    df['purchase_date'] = df['purchase_date'].str.slice(0, 19)
    df['purchase_date'] = pd.to_datetime(df['purchase_date'],
                                         format='%Y-%m-%d %H:%M:%S')

    # Encode categorical columns
    bool_map = {'Y': 1, 'N': 0}
    three_map = {'A': 0, 'B': 1, 'C': 2}
    conversions = [
        ('authorized_flag', bool_map, 'uint8'),
        ('category_1', bool_map, 'uint8'),
        ('category_3', three_map, 'float32'), #has NaNs so have to use float
    ]
    for col, mapper, new_type in conversions:
        df[col] = df[col].map(mapper).astype(new_type)
        
    # Clean up
    gc.collect()

# Preprocess the transactions data
preprocess_trans_data(hist_trans)
preprocess_trans_data(new_trans)

## Merge merchants with transactions data

For each transaction, we need to add the information about the merchant involved in that transaction to the transactions dataframe.

In [21]:
# Merge transactions with merchants data
hist_trans = pd.merge(hist_trans, merchants, on='merchant_id')
new_trans = pd.merge(new_trans, merchants, on='merchant_id')
del merchants
gc.collect()

hist_trans.sample(10)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,merchant_group_id,numerical_1,numerical_2,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4
23962616,1,17766,181,0,0,0.0,793,143601.0,-4,-0.581616,2017-10-04 16:38:08,5.0,5,38,8080,-0.047556,-0.057471,3,4,1.190476,0.887324,3,1.052632,0.840000,6,1.086957,0.938547,12,0
19320397,1,104674,333,0,2,2.0,818,81208.0,-4,0.026387,2017-10-01 14:30:06,5.0,21,12,5884,-0.007896,-0.037641,2,3,0.833333,0.939963,3,0.671141,0.869792,6,0.641026,0.855131,12,1
17874768,1,40799,60,0,0,0.0,367,140243.0,-1,-0.616793,2018-01-08 19:53:24,1.0,16,16,2084,0.051593,-0.057471,1,1,0.917431,0.937217,3,0.917431,0.923157,6,0.900901,0.906187,12,0
2007759,1,274709,69,0,1,1.0,879,854.0,-2,-0.663661,2017-12-20 16:44:00,1.0,9,29,35,-0.047556,-0.057471,4,4,0.144300,0.078708,3,0.118765,0.063069,6,0.116686,0.061653,7,1
4365438,1,299526,17,0,0,0.0,705,289813.0,-3,-0.328269,2017-11-12 00:17:30,4.0,22,33,90167,0.130913,0.061508,0,0,1.030928,1.032623,3,1.041667,1.059276,6,1.010101,1.047233,12,1
26662078,1,277256,283,0,0,0.0,560,38788.0,-2,-0.733745,2017-12-15 16:05:04,1.0,9,34,27646,-0.057471,-0.057471,3,2,1.052632,1.027674,3,1.075269,1.051061,6,1.098901,1.075012,12,0
2321577,1,134651,69,0,0,0.0,879,854.0,-3,-0.471923,2017-11-03 17:01:25,1.0,9,29,35,-0.047556,-0.057471,4,4,0.144300,0.078708,3,0.118765,0.063069,6,0.116686,0.061653,7,1
5058314,1,53385,69,0,1,1.0,683,50029.0,-3,-0.549940,2017-11-02 23:19:31,1.0,9,34,35,105.585838,104.901711,0,0,0.925926,0.917322,3,0.909091,0.915565,6,0.884956,0.894929,12,1
11610944,1,30432,-1,1,1,1.0,511,90919.0,-4,-0.745405,2017-10-30 10:56:55,NaN,-1,7,35,-0.057471,-0.057471,2,2,0.862069,0.845911,3,0.869565,0.845490,6,1.000000,0.918818,12,1
8736575,1,293667,25,0,0,0.0,528,153077.0,-9,-0.725119,2017-05-08 20:35:20,3.0,7,25,14042,-0.057471,-0.057471,2,2,0.854701,0.858472,3,0.751880,0.762652,6,0.793651,0.816122,12,1


## Feature Engineering (for Transactions)

First let's one-hot `category_2` and `category_3`, while keeping the original column (since they may be ordinal).

In [22]:
def add_one_hot(df, col):
    """Add one-hot columns"""
    ucats = df[col].unique() #unique categories
    ucats = ucats[~np.isnan(ucats)] #that aren't nan
    for oh_col in ucats:
        df[col+'_'+str(int(oh_col))] = (df[col] == oh_col).astype('uint8')
        
for df in [hist_trans, new_trans]:
    add_one_hot(df, 'category_2')
    add_one_hot(df, 'category_3')    

First, let's create some features involving when the transactions occurred.

In [23]:
# Compute features from purchase time/date
for df in [hist_trans, new_trans]:
    tpd = df['purchase_date']
    ref_date = np.datetime64('2017-09-01')
    one_hour = np.timedelta64(1, 'h')
    df['purchase_hour'] = tpd.dt.hour.astype('uint8')
    df['purchase_day'] = tpd.dt.dayofweek.astype('uint8')
    df['purchase_week'] = tpd.dt.weekofyear.astype('uint8')
    df['purchase_month'] = tpd.dt.month.astype('uint8')
    df['purchase_weekend'] = (df['purchase_day'] >=5 ).astype('uint8')
    df['purchase_time'] = ((tpd - ref_date) / one_hour).astype('float32')
    df['ref_date'] = ((tpd - pd.to_timedelta(df['month_lag'], 'M')
                          - ref_date ) / one_hour).astype('float32')

    # Time sime first active
    tsfa = pd.merge(df[['card_id']], 
                    cards[['first_active_month']].copy().reset_index(),
                    on='card_id', how='left')
    df['time_since_first_active'] = ((tpd - tsfa['first_active_month'])
                                     / one_hour).astype('float32')

    # TODO: whether the day is a holiday or week before a holiday or something?

    # Clean up
    del tsfa
    del df['purchase_date']
    gc.collect()

Now we can convert `first_active_month` to months.

In [24]:
cards['first_active_month'] = (12*(cards['first_active_month'].dt.year-2011) + 
                               cards['first_active_month'].dt.month).astype('float32')

In [25]:
hist_trans.sample(10)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,category_2,state_id,subsector_id,merchant_group_id,numerical_1,numerical_2,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_2_1,category_2_5,category_2_2,category_2_3,category_2_4,category_3_0,category_3_1,category_3_2,purchase_hour,purchase_day,purchase_week,purchase_month,purchase_weekend,purchase_time,ref_date,time_since_first_active
8092394,1,140068,69,0,0,0.0,705,222856.0,-1,-0.725645,1.0,9,33,31972,0.130913,-0.057471,0,0,0.990099,0.988839,3,0.980392,0.988420,6,0.970874,0.981404,12,1,1,0,0,0,0,1,0,0,19,4,1,1,0,3043.225342,3773.710205,5995.225098
22698219,1,95462,19,0,0,0.0,307,177312.0,-11,-0.671775,1.0,9,19,43659,-0.047556,-0.047556,2,2,0.917431,0.930767,3,0.900901,0.897258,6,0.909091,0.846395,12,0,1,0,0,0,0,1,0,0,18,4,10,3,0,-4181.851074,3853.484131,3858.149170
21091284,1,241572,30,0,1,1.0,706,70060.0,-1,-0.584621,3.0,17,26,64747,-0.057471,-0.057471,4,3,0.990099,1.047109,3,1.123596,1.215662,6,1.149425,1.286842,12,1,0,0,0,1,0,0,1,0,16,2,2,1,0,3160.842773,3891.327881,9736.842773
24401921,1,201715,143,0,1,1.0,560,10883.0,-11,-0.727599,5.0,5,34,60514,-0.057471,-0.057471,3,2,1.075269,0.998455,3,1.136364,1.081506,6,1.111111,1.082956,12,1,0,1,0,0,0,0,1,0,16,1,12,3,0,-3919.019287,4116.315918,1912.980835
21091780,1,231718,251,0,0,0.0,278,35565.0,-11,-0.579362,3.0,8,37,81354,-0.037641,-0.037641,2,2,1.000000,1.050607,3,1.000000,1.072536,6,0.970874,1.032733,12,1,0,0,0,1,0,1,0,0,0,2,9,3,0,-4415.466797,3619.868408,1416.533325
28863288,1,39583,212,0,4,2.0,792,234052.0,-7,1.281671,4.0,4,9,52951,-0.057471,-0.057471,3,4,0.869565,1.076087,3,0.934579,1.125000,6,0.990099,1.196375,12,1,0,0,0,0,1,0,0,1,14,1,30,7,0,-897.581970,4215.812988,5678.417969
29851104,1,100391,286,0,0,0.0,411,114864.0,-4,-0.641722,3.0,3,21,75704,-0.057471,-0.057471,3,4,0.980392,1.005236,3,1.176471,1.081690,6,1.265823,1.207547,12,1,0,0,0,1,0,1,0,0,2,1,42,10,0,1106.000000,4027.939941,11354.000000
20756956,1,67777,325,0,0,0.0,705,18957.0,-3,-0.659754,1.0,9,33,35,1.152148,1.152148,1,1,0.917431,0.934162,3,0.900901,0.925195,6,0.854701,0.891355,12,0,1,0,0,0,0,1,0,0,22,5,40,10,1,886.214722,3077.669678,2374.214844
16386577,1,80343,1,0,0,0.0,80,117209.0,-7,-0.732858,1.0,12,37,15801,-0.057471,-0.057471,4,4,0.763359,0.754491,3,0.689655,0.676510,6,0.609756,0.609432,12,1,1,0,0,0,0,1,0,0,17,1,30,7,0,-894.174438,4219.220703,3521.825439
21276720,1,158095,25,0,0,0.0,2,195903.0,-5,-0.656749,3.0,7,20,49947,-0.047556,-0.047556,3,4,2.439024,0.967742,3,3.448276,1.165049,6,2.857143,1.000000,12,1,0,0,0,1,0,1,0,0,16,1,22,5,0,-2239.315186,1413.109741,2848.684814


## Feature Engineering (aggregations)

Now we engineer features for each card ID by applying aggregation functions to each card's transactions data.  First we need to group the transactions by `card_id`.

In [26]:
# Group transactions by card id
hist_trans = hist_trans.groupby('card_id', sort=False)
new_trans = new_trans.groupby('card_id', sort=False)

First let's define some custom aggregation functions:

In [27]:
def entropy(series):
    """Categorical entropy"""
    probs = series.value_counts().values.astype('float32')
    probs = probs / np.sum(probs)
    probs[probs==0] = np.nan
    return -np.nansum(probs * np.log2(probs))

def mean_diff(series):
    """Mean difference between consecutive items in a series"""
    ss = series.sort_values()
    return (ss - ss.shift()).mean()

def period(series):
    """Period of a series (max-min)"""
    return series.max() - series.min()

def mode(series):
    """Most common element in a series"""
    tmode = series.mode()
    if len(tmode) == 0:
        return np.nan
    else:
        return tmode[0]

And then we can perform the aggregations.

In [28]:
# Aggregations to perform for each predictor type
binary_aggs = ['sum', 'mean', 'nunique']
categorical_aggs = ['nunique', entropy, mode]
continuous_aggs = ['min', 'max', 'sum', 'mean', 'std', 'skew', mean_diff, period]

# Aggregations to perform on each column
aggs = {
    'authorized_flag':             binary_aggs,
    'city_id':                     categorical_aggs,
    'category_1':                  binary_aggs,
    'installments':                continuous_aggs,
    'category_3':                  categorical_aggs + ['mean'], #might be ordinal
    'category_3_0':                ['mean'],
    'category_3_1':                ['mean'],
    'category_3_2':                ['mean'],
    'category_2_1':                ['mean'],
    'category_2_2':                ['mean'],
    'category_2_3':                ['mean'],
    'category_2_4':                ['mean'],
    'category_2_5':                ['mean'],
    'merchant_category_id':        categorical_aggs,
    'merchant_id':                 categorical_aggs,
    'month_lag':                   continuous_aggs,
    'purchase_amount':             continuous_aggs,
    'purchase_time':               continuous_aggs + ['count'],
    'purchase_hour':               categorical_aggs + ['mean'],
    'purchase_day':                categorical_aggs + ['mean'],
    'purchase_week':               categorical_aggs + continuous_aggs,
    'purchase_month':              categorical_aggs + continuous_aggs,
    'purchase_weekend':            binary_aggs,
    'ref_date':                    continuous_aggs,
    'time_since_first_active':     continuous_aggs,
    'category_2':                  categorical_aggs + ['mean'], #also might be ordinal
    'state_id':                    categorical_aggs,
    'subsector_id':                categorical_aggs,
    'merchant_group_id':           categorical_aggs,
    'numerical_1':                 continuous_aggs,
    'numerical_2':                 continuous_aggs,
    'most_recent_sales_range':     categorical_aggs + ['mean'], #ordinal?
    'most_recent_purchases_range': categorical_aggs + ['mean'], #orindal?
    'avg_sales_lag3':              continuous_aggs,
    'avg_purchases_lag3':          continuous_aggs,
    'active_months_lag3':          continuous_aggs,
    'avg_sales_lag6':              continuous_aggs,
    'avg_purchases_lag6':          continuous_aggs,
    'active_months_lag6':          continuous_aggs,
    'avg_sales_lag12':             continuous_aggs,
    'avg_purchases_lag12':         continuous_aggs,
    'active_months_lag12':         continuous_aggs,
    'category_4':                  binary_aggs,
}

# Perform each aggregation
for col, funcs in aggs.items():
    for func in funcs:
        
        # Get name of aggregation function
        if isinstance(func, str):
            func_str = func
        else:
            func_str = func.__name__
            
        # Name for new column
        new_col = col + '_' + func_str
            
        # Compute this aggregation
        cards['hist_'+new_col] = hist_trans[col].agg(func).astype('float32')
        cards['new_'+new_col] = new_trans[col].agg(func).astype('float32')

In [29]:
def remove_noninformative(df):
    """Remove non-informative columns (all nan, or all same value)"""
    for col in df:
        if df[col].isnull().all():
            print('Removing '+col+' (all NaN)')
            del df[col]
        elif df[col].nunique()<2:
            print('Removing '+col+' (only 1 unique value)')
            del df[col]

remove_noninformative(cards)
gc.collect()

Removing new_authorized_flag_mean (only 1 unique value)
Removing new_authorized_flag_nunique (only 1 unique value)
Removing hist_active_months_lag3_max (only 1 unique value)
Removing hist_active_months_lag6_max (only 1 unique value)


3430

In [30]:
max_len = max([len(e) for e in cards.columns])
fmt_str = "{:>"+str(max_len)+"}  {}"
print(fmt_str.format('COLUMN', 'DATATYPE'))
for col in cards:
    print(fmt_str.format(col, cards[col].dtype))

                                  COLUMN  DATATYPE
                      first_active_month  float32
                               feature_1  uint8
                               feature_2  uint8
                               feature_3  uint8
                                  target  float32
                hist_authorized_flag_sum  float32
                 new_authorized_flag_sum  float32
               hist_authorized_flag_mean  float32
            hist_authorized_flag_nunique  float32
                    hist_city_id_nunique  float32
                     new_city_id_nunique  float32
                    hist_city_id_entropy  float32
                     new_city_id_entropy  float32
                       hist_city_id_mode  float32
                        new_city_id_mode  float32
                     hist_category_1_sum  float32
                      new_category_1_sum  float32
                    hist_category_1_mean  float32
                     new_category_1_mean  float32
     

In [31]:
cards.info()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 325540 entries, 186538 to 172900
Columns: 443 entries, first_active_month to new_category_4_nunique
dtypes: float32(440), uint8(3)
memory usage: 549.8 MB


In [32]:
# Save to feather file
cards.reset_index(inplace=True)
cards.to_feather('card_features_all.feather')

# Read back in with:
# cards = pd.read_feather('../input/elo-feature-engineering/card_features.feather')
# cards.set_index('card_id', inplace=True)
# You can read in the data from another kernel by setting this kernel as a data source:
# https://www.kaggle.com/rtatman/importing-data-from-a-kernel